# Patent data integration and EDA

This is an exploratory analysis of PATSTAT application data involving GB-based inventors and applicants.

For more information about PATSTAT data check [here](https://www.epo.org/searching-for-patents/business/patstat.html#tab-1) and for more informatin about patent analysis in general go [here](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/463319/The_Patents_Guide_2nd_edition.pdf) 

### Activities

* Load data (currently saved as a pickled dict where every element is a dataframe with information about an application)
* Integrate data into a smaller set of tables for analysis
* Explore data: 
  * What are the variables in the data?
  * What are the missing and present values?
  * What do the data capture *legally*?
* Carry out an initial exploratory analysis
  * What are the activity trends?
  * Who are the top patenters (organisations)
  * What are the top patenters (sectors / places)
* What can we find about AI?


### Outputs

* A data dictionary and cleaned dataset


## 0. Preamble

In [ ]:
%run notebook_preamble.ipy

In [ ]:
import pandas_profiling as pp

### 1. Load data

We have stored the data in a pickled file with a list of dictionaries containing various information.

In [ ]:
with open('../data/raw/20_6_2019_patent_outputs.p','rb') as infile:
    pdict = pickle.load(infile)

In [ ]:
type(pdict)

In [ ]:
len(pdict)

It contains 8 dictionaries

In [ ]:
#We have a quick look inside

#Loop over items in the dict
for k,v in pdict.items():
    
    print(k.upper())
    print(len(k)*'=')
    print('\n')
    
    print(f'number of observations: {len(v)}')
    
    print('\n')

    
    print(v.head())
    
    print('\n')
        
    print('Columns')
    print('======')
    
    print(v.columns)
    
    
    print('\n \n')

In [ ]:
# Create a dictionary for patent outputs

# for k,v in pdict.items():
    
#     print(f'* **{k}**:')
#     #print(f' * description:')
#     print(f' * length:{len(v)}')
    

## EDA of patent output contents

### Person - applications

The person application dataframe contains information about GB inventors or applications - they are the seed for our patent analysis in the mapping innovation in Scotland project.

In [ ]:
pp.ProfileReport(pdict['person_appln'])

### A couple of questions

* Which name and ID do we use?
* What's up with all those missing addresses?



In [ ]:
papp = pdict['person_appln']

In [ ]:
papp['han_name'].value_counts()[:20]

In [ ]:
papp['psn_name'].value_counts()[:20]

The han_name seems to be missing universities - they are not in Orbis?

In [ ]:
papp.loc[papp['psn_name']=='UNIVERSITY OF CAMBRIDGE']['han_name'].head()

Let's use the `psn_name` as this is the 'official' patstat standardised name

### What's up with the missing addresses?

In [ ]:
papp['person_address'].isna().mean()

In [ ]:
person_address_lookup = {row['psn_name']:row['person_address'] for ind,row in papp.dropna(axis=0,subset=['person_address']).iterrows()}

In [ ]:
# How many of the names with missing addresses are in this lookup?

names_missing_add = papp.loc[papp['person_address'].isna()]['psn_name']

names_missing_add.value_counts()[:10]

Interesting - many of the orgs with missing addresses are 'big organisations'

In [ ]:
len(set(names_missing_add)-set(person_address_lookup.keys()))

There are still 218K with missing addresses - they are not in the person name - address lookup

### Do organisations have a single address, and how do we interpret it?

In [ ]:
#This groups the data by organisations and creates a list of addresses. Do we have multiple addresses per name or only one?

grouped_addresses = papp.groupby('psn_name')['person_address'].apply(lambda x: set(list(x)))

In [ ]:
grouped_addresses

Ok - so there seems to be a lot of duplication here. One way to manage this would be to focus on harmonised names

In [ ]:
pd.Series([len(x) for x in grouped_addresses]).value_counts()[:5]

There are lots of names with multiple addresses - we will need to allocate them at the patent level. Also need to decide what to do with missing values



In [ ]:
grouped_addresses.loc[[(len(x)>50) for x in grouped_addresses]][:10]

The organisations with many addresses are big organisations or very common names. Do people with very common names have a single person id or many?

In [ ]:
grouped_addresses['UNILEVER']

What a mess! The addresses are totally unstandardised

We can at least extract their postcodes using nslp

#### Extract postcodes using NSPL, the postcode lookup

In [ ]:
#Load it
#TODO - remove hardcoded path

nspl = pd.read_csv('/Users/jmateosgarcia/Desktop/data/nspl/NSPL_FEB_2018_UK.csv')

In [ ]:
#Create a list of lowercase postcodes. We will focus on the first three letters as this will speed up the analysis
postcodes = list(set(nspl['pcds'].apply(lambda x: x.lower().split(' ')[0])))

In [ ]:
#Lowercase the patent applications too
papp['address_lower'] = papp['person_address'].apply(lambda x: x.lower().split(' ') if pd.isnull(x)==False else np.nan)

In [ ]:
#Now extract the postcodes from the lowercase addresses (if present)
papp['uk_postcode'] = [set(x) & set(postcodes) if type(x)==list else np.nan for x in papp['address_lower']]

In [ ]:
#And now we want to extract full postcodes for those people where we found the 3-digit ones
# This is a faff

In [ ]:
%%time

#Store full postcodes here

full_postcode_store = []

#We will loop over rows

for ind,row in papp.iterrows():
    
    #If the postcode is nan that means we append a nan to our store
    
    if type(row['uk_postcode'])!=set:
        full_postcode_store.append(np.nan)
    
    else:
        
    #If we have a postcode, we extract it together with the address

        pc = list(row['uk_postcode'])
        add = row['address_lower']

    # There were addresses with no postcodes - empty set. In there was at least one we will try to extract the string after it

        if (len(pc)>0):

            #Index for the postcode. Note that this is assuming that we had a unique postcode per address
            ind = add.index(pc[0])

            #print(ind+1)
            #print(len(add))
            
            #In some cases we have the first three digit of the postcode at the end of the address. In that case we append those.
            if ind+1 < len(add):
                
                #Join the postcode with the string immediately after.
                #Note that in some cases this might append non-postcode strings. These won't be matched later on.
                
                
                out = ' '.join([pc[0],add[ind+1]])
                full_postcode_store.append(out)

            #If we didn't have a full postcode we append the three digits extracted before.
            else:
                full_postcode_store.append(pc[0])
        
        # If the set was empty, this means we have no postcode
        else:
            full_postcode_store.append(np.nan)
    


In [ ]:
papp['uk_postcode_long'] = full_postcode_store

Let's merge with TTWAs

In [ ]:
#Lowercase the postcodes as before

nspl['pcds_lower'] = nspl['pcds'].apply(lambda x: x.lower())

In [ ]:
#Do the merge

papp_geo = pd.merge(papp,nspl[['pcds_lower','laua','ttwa']],left_on='uk_postcode_long',right_on='pcds_lower',how='outer')

In [ ]:
# Add TTWA names
# TODO: remove hardcoded path

#Load the lookup
ttwa_names = pd.read_csv('/Users/jmateosgarcia/Desktop/data/nspl/Documents/TTWA names and codes UK as at 12_11 v5.txt',delimiter='\t')

#Create the dict (is there a better way to do this?)
ttwa_names_lookup = {x['TTWA11CD']:x['TTWA11NM'] for ind,x in ttwa_names.iterrows()}

In [ ]:
#Map
papp_geo['ttwa_name'] = papp_geo['ttwa'].map(ttwa_names_lookup)

In [ ]:
#Here we go. Looking good
papp_geo['ttwa_name'].value_counts()[:10]

### Do individuals with very common names have different ids?

In [ ]:
papp.loc[papp['psn_name']=='BAKER, MATTHEW'][['person_id','person_name','psn_name','psn_id','han_name','han_id','person_address']].sort_values('han_id')

It is unclear what is the link between ids and person names. We will need to match at the patent application id level and decide what we do with missing addresses. Somehow allocate missing addresses randomly based on address distributions for persons with the same name (or id?)

#### Add flags for wheter a person is applicant or inventor

In [ ]:
papp_geo['is_inventor'],papp_geo['is_applicant'] = [[x>0 for x in papp_geo[var]] for var in ['invt_seq_nr','applt_seq_nr']]

#### Impute TTWAs where this is missing (TODO)

### Conclusion: create a table that we can merge with the patent applications later

We will group various bits of information by the patent id (which becomes the index we will use for merging). They include:

* Inventor (`invt_seq_nr` different from zero) names, ids addresses and TTWAs
* Applicant (`applt_seq_nr` different from zero) names, ids, addresses and TTWAS

To do this, I will create a simple function `make_person_metadata`


In [ ]:
def make_person_metadata(df,metadata,name,application_id='appln_id'):
    '''
    This function creates patent application level metadata about the persons involved.
    
    In order to produce metadata about applicants and inventors we will filter the df beforehand using the invt_seq_nr and applt_seq_nr variables
    
    Arguments:
        -df is the patent person df with the relevant information
        -metadata is the list of variables that we want to aggreate for each patent
        -name is the prefix we will use to label the data (eg inv, appl)
        -application_id is the application identifier
        
    Output:
        -A df where every row is a patent application and the columns contain the metadat
    
    '''
    
    #Generate the metadata for each variable and output
    out = pd.concat([df.groupby(application_id)[var].apply(lambda x: list(x)) for var in metadata],axis=1)
    
    out.rename(columns = {x:name+'_'+x for x in out.columns},inplace=True)
    
    return(out)
    
    
    

In [ ]:
#These are the metadata variables of interest
meta_vars = ['psn_name','psn_id','psn_sector','person_address','uk_postcode_long','ttwa','ttwa_name']

#This is a list with a df of 'person applicants' and a df of person inventors
subset_dfs = [papp_geo.loc[papp_geo[var]==True] for var in ['is_applicant','is_inventor']]

#This extracts the metadata for applicant and inventor metadata sets 
pat_person_meta = pd.concat([make_person_metadata(df,metadata=meta_vars,name=name) for df,name in zip(subset_dfs,
                                                                                                        ['appl','inv'])],axis=1)

In [ ]:
pat_person_meta.head()

Note - some of these patents have missing applicants or inventors because eg these might be based outside of the uk

#### What are the missing addresses of inventor / applicant dfs

In [ ]:
subset_dfs[0]['person_address'].isna().mean()

## appln

The `appln` df contains information about patent applications, such as their year and their 'family' (the invention they refer to).

In [ ]:
app = pdict['appln']

In [ ]:
pp.ProfileReport(app)

In [ ]:
app.columns

### A couple of things to explore

### Interpretation of dates

In [ ]:
# What is the relation between filing year and publication year?

100*np.mean(app['earliest_filing_year']<=app['earliest_publn_year'])

This is as expected - patents are filed with the patent office, after which they are published

### Interpretation of patent families - do they tend to be in the same jurisdiction or different ones?

In [ ]:
app['docdb_family_id'].value_counts()[:10]

We will check the jurisdictions for the patent with the biggest family

In [ ]:
app.loc[app['docdb_family_id']==9905751]['appln_auth'].value_counts()

Applications in multiple jurisdictions suggesting that a focus on families helps us to avoid double counting.

Read an easy to understand explanation in [Wikipedia](https://en.m.wikipedia.org/wiki/Priority_right)

In [ ]:
app.loc[app['docdb_family_id']==9905751]['nb_citing_docdb_fam'].head()

All patents in a family receive the same number of citations. Another reason to focus on citation

#### Conclusion: create an app_subset with variables of interest

In [ ]:
app.columns

In [ ]:
my_vars = ['appln_id','appln_nr','ipr_type','granted', 'appln_auth','appln_filing_year','earliest_publn_year',
          'docdb_family_id','inpadoc_family_id','nb_citing_docdb_fam']


app_subset = app[my_vars].set_index('appln_id')

app_subset.head()

In [ ]:
len(app_subset)

In [ ]:
len(set(app_subset.index))

## appln_abstract

In [ ]:
abst = pdict['appln_abstr']

abst

In [ ]:
pp.ProfileReport(abst)

In [ ]:
abst_length = pd.Series([len(x) for x in abst['appln_abstract']])

abst_length.describe()

Almost all patents are in English. Some of them are incredibly long!

### Out of curiosity: 

* How many of them mention finance?

In [ ]:
np.sum(['financ' in x for x in abst['appln_abstract']])

* And how many mention machine learning?

In [ ]:
np.sum(['machine learning' in x for x in abst['appln_abstract']])

This looks quite low - let's see if we can match the ai patents later and see what happens

## appln_techfield

In [ ]:
techfield = pdict['appln_techn_field']

techfield.head()

In [ ]:
pp.ProfileReport(techfield)

Each patent is allocated a set of technology fields (with weights).

In [ ]:
pdict.keys()

## techn_field_ipc

This is a lookup table

In [ ]:
tf_lookup = pdict['tls901_techn_field_ipc']

tf_lookup.head()

In [ ]:
pp.ProfileReport(tf_lookup)

### Match the techn fields with the previous field (so we can do some interpretable exploration)

In [ ]:
techfield_labelled = pd.merge(techfield,tf_lookup.drop_duplicates('techn_field'),left_on='techn_field_nr',right_on='techn_field_nr')

In [ ]:
techfield_labelled.head()

In [ ]:
techfield_labelled.groupby('techn_field')['weight'].sum().sort_values(ascending=False)

In [ ]:
# I need to group the fields by patent ids

tf_meta_vars = ['weight','techn_field_nr','techn_field']

#I use the same function that I defined before (it's quite generic!)
tech_grouped = make_person_metadata(techfield_labelled,metadata=tf_meta_vars,application_id='appln_id',name='tf')


In [ ]:
tech_grouped.head()

## tls902_ipc_nace2

This is a lookup between ip codes and nace. Won't be very useful for us as we don't have the nace codes...yet

In [ ]:
ipc_nace_lookup = pdict['tls902_ipc_nace2']

In [ ]:
ipc_nace_lookup.head()

In [ ]:
pp.ProfileReport(ipc_nace_lookup)

## NUTS lookup (for completeness)

In [ ]:
nuts_lookup = pdict['tls904_nuts']

nuts_lookup.head()

In [ ]:
pp.ProfileReport(nuts_lookup)

## Combine sources

Here we will combine all the tables so far:

* `app_subset` has the applications
* `pat_person_meta` has the persons
* `abstr` has the abstracts
* `techfield_labelled` has the patent tech fields (with labels)

The other dfs are not massively relevant

In [ ]:
processed_dfs = [app_subset,pat_person_meta,abst,
         tech_grouped]

In [ ]:
for name,df in zip(['appl','person','abstract','field'],processed_dfs):
    
    print(name)
    print('===')
    
    print(len(df))
    
    print('\n')

In [ ]:
pat = pd.concat(processed_dfs,axis=1,join='outer')

In [ ]:
pat.head()

## Create data dictionary

In [ ]:
pat.reset_index(drop=False,inplace=True)

print('|name|type|observations|')
print('|----|----|----|')

for c in pat.columns:
    
    print(f'|{c}|{type(pat[c].iloc[0])}|   |')

### Are there any ML patents in here?

Load the IPO patents (downloaded from [here](https://www.gov.uk/government/publications/artificial-intelligence-a-worldwide-overview-of-ai-patents)

In [ ]:
ml_ids = list(pd.read_csv('../data/external/AI-raw-data.csv',header=None)[0])

In [ ]:
#In order to match these patents with our data we need to create a new id that combines granting authority code and publication number 
pat['raw_ids'] = [x+y for x,y in zip(pat['appln_auth'],pat['appln_nr'])]

In [ ]:
#What's the overlap between both groups?
uk_ai_pats = set(list(pat['raw_ids'])) & set(ml_ids)

len(uk_ai_pats)

1012 - not so bad!

In [ ]:
pat['is_ai_ipo'] = [x in uk_ai_pats for x in pat['raw_ids']]

In [ ]:
pat.to_csv(f'../data/processed/{today_str}_patent_table.csv',compression='gzip')